`mamba install arraylake dask distributed odc-geo xarray`

In [ ]:
import arraylake as al
import xarray as xr

# To run on coiled, you'll need to set token=TOKEN here,
# and then set that token as coiled.Cluster(environ={"ARRAYLAKE_TOKEN": TOKEN})
alclient = al.Client()
alclient

In [ ]:
import odc.geo.xr
from odc.geo.geobox import GeoBox

outbox = GeoBox.from_bbox(
    (-130.24, 21.58, -64.12, 52.88),
    crs="epsg:4326", resolution=0.01
)
outbox

## Read data

In [ ]:
alclient = al.Client()
alclient

In [ ]:
repo = alclient.get_repo("earthmover-public/nasa-nex-gdm")
nex = (
    repo.to_xarray(decode_coords="all", chunks=None)
    .chunk(time=50, x=200, y=200)
)
nex

Not sure if this next cell matters.

In [ ]:
import numpy as np

def calc_rel_hum(ds):
    T = (ds.tmin + ds.tmax) / 2
    es = np.exp(17.67 * T / (T + 243.5))
    e = np.exp(17.67 * ds.tdew / (ds.tdew + 243.5))
    return e / es * 100

nex["rh"] = calc_rel_hum(nex)
nex

## Read data

In [ ]:
nex_reproj = (
    # subset to 10 year span
    nex.sel(time=slice("2013", "2022"))
    # reproject to epsg:4326
    .odc.reproject(how=outbox, resampling="average", chunks=(200, 200))
    # ascending latitudes
    .isel(latitude=slice(None, None, -1))
    # chunk to uniform chunks for zarr
    .chunk(time=50, latitude=600, longitude=600)
)
nex_reproj

In [ ]:
# chunk to expected chip
CHUNK_SIZE = (50, 100, 100)
encoding = {var: {"chunks": CHUNK_SIZE} for var in nex_reproj}
nex_reproj.to_zarr(ADD_A_ZARR_ON_S3_URI, group="nex/4326", mode="w", encoding=encoding)